# Web Scrapping estatísticas superliga feminina 2018-2019

In [1]:
# Importando os pacotes necessários
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd

### Função Scrapper - Tabela de Jogos

In [2]:
def scrapper_tabela_jogos(url):
    
    # Coletando a página
    with request.urlopen(url) as website:
        page = website.read()

    soup = BeautifulSoup(page, 'html.parser')
    
    # Coletando dados
    rows_data_local = soup.find_all('div', class_="t-col t-col-2 t-col-xs-3 t-col-sm-3 t-hidden-xs t-hidden-sm t-hidden-md Calendar_DIV_Column")
    rows_times = soup.find_all('div', class_="t-col t-col-2 t-col-sm-3 t-col-md-2 t-hidden-xs Calendar_DIV_Column")
    rows_placar = soup.find_all('div', class_="t-col t-col-1 t-col-xs-4 t-col-sm-3 t-col-md-2 Calendar_DIV_Column")
    
    # Coletando os IDs dos jogos
    ids = []

    for row in rows_data_local:
        result = row.find_all('p', class_='Calendar_p_TextRow Calendar_p_TextRow_Italic')
        id_jogo = str(result[0]).split('MatchStatistics.aspx?mID=')[1].split('&')[0]
        ids.append(int(id_jogo))
        
    
    
    # Coletando todos as datas e locais e armazenando em listas
    datas = []
    local = []

    for row in rows_data_local:
        result = row.find_all('span')
        datas.append(result[0].string)
        local.append(result[1].string)

        
    # Coletando os jogos 
    mandante = []
    visitante = []

    for index in range(len(rows_times)):
        time = rows_times[index].find_all('span')
        if index%2 == 0:
            mandante.append(time[0].string)
        else:
            visitante.append(time[0].string)


    # Coletando os placares
    sets_mandante = []
    sets_visitante = []

    for row in rows_placar:
        result = row.find_all("span")
        if len(result) == 3:
            sets_mandante.append(int(result[0].string))
            sets_visitante.append(int(result[2].string))
        else:
            sets_mandante.append(None)
            sets_visitante.append(None)
    
    
    # Gerando Rodada e Turnos
    rodada = []
    turno = []

    t = 1
    rt = 0
    rq = 0
    rs = 0


    for i in range(153):

        if i < 132:
            if i%6==0:
                t = t*(-1)
            if t > 0:
                turno.append('Primeiro Turno')
            else:
                turno.append('Segundo Turno')


            if i%12 == 0:
                rt = rt+1
            rodada.append(rt)

        elif i < 144:
            turno.append('quartas')
            if i%4 == 0:
                rq = rq+1
            rodada.append(rq)

        elif i < 150:
            turno.append('semis')
            if i%2==0:
                rs=rs+1
            rodada.append(rs)

        else:
            turno.append('finais')
            rodada.append(i-149)
        
        
    # Criando DataFrame do Pandas os dados
    df = pd.DataFrame({
        'id_jogo': ids,
        'data_jogo': datas,
        'local_jogo': local,
        'mandante': mandante,
        'sets_mandante': sets_mandante,
        'sets_visitante': sets_visitante,
        'visitante': visitante,
        'rodada': rodada,
        'turno': turno,
    })
    
    df = df.sort_values(by=['turno', 'rodada']).set_index('id_jogo')
    return df
    

### Função Scrapper estatísticas

In [3]:
def estatisticas_jogo(id_jogo):
    
    # Coletando dados da página
    url = f"http://cbv-web.dataproject.com/MatchStatistics.aspx?mID={id_jogo}"

    with request.urlopen(url) as website:
        page = website.read()

    soup = BeautifulSoup(page, 'html.parser')
    
    # Coletando as tabelas da página
    tables = soup.find_all('table', class_='rgMasterTable')
    stats_mandante = tables[0]
    stats_visitante = tables[1]
    
    # Time mandante
    mandante = soup.find_all('div', class_='t-col t-col-12 RPL_ColumnPadding_Zero DIV_GenericContentBackground')[0]
    mandante = mandante.find_all('span')[0].string
    
    # Time visitante
    visitante = soup.find_all('div', class_='t-col t-col-12 RPL_ColumnPadding_Zero DIV_GenericContentBackground')[1]
    visitante = visitante.find_all('span')[0].string
    
    # Coletando os dados
    tabelas = []

    for tabela in [stats_mandante, stats_visitante]:

        # Coletando o body da tabela
        tbody = tabela.find_all('tbody')

        # Coletando as linhas da tabela
        rows = tbody[0].find_all('tr')

        linhas = []
        for row in rows:
            data = row.find_all('span')

            dados = []
            for datum in data:
                dados.append(datum.string)

            linhas.append(dados)

        tabelas.append(linhas)

    # Cabeçalho das tabelas
    th = ['#', 'Jogadora', 'set_1', 'set_2', 'set_3', 'set_4', 'set_5', 'Pontos_Tot', 'BP', 'V-P', 'a', 'Saques_Tot', 'Saques_Err', 'Saques_Ace', 'Rec_Tot', 'Rec_Err', 'Rec_%Pos', 'Rec_%_Exc.', 'b', 'c', 'At_Tot', 'At_Err', 'At_Blk', 'At_Exc.', '% Exc. ', 'd', 'Block_Pts', 'e']
    
    # Removendo a linha de totais
    totais_mandante = tabelas[0].pop()
    totais_visitante = tabelas[1].pop()
    
    totais = [totais_mandante, totais_visitante]
    
    df_mandante = pd.DataFrame(tabelas[0], columns=th).drop(['a', 'b', 'c', 'd', 'e'], axis=1)
    df_mandante['id_jogo'] = id_jogo
    df_mandante = df_mandante.set_index(["id_jogo", "Jogadora"])
    df_mandante['time'] = mandante
    
    df_visitante = pd.DataFrame(tabelas[1], columns=th).drop(['#', 'a', 'b', 'c', 'd', 'e'], axis=1)
    df_visitante['id_jogo'] = id_jogo
    df_visitante = df_visitante.set_index(["id_jogo", "Jogadora"])
    df_visitante['time'] = visitante
    
    df_totais = pd.DataFrame(totais, columns=th).drop(['#', 'a', 'b', 'c', 'd', 'e'], axis=1)
    df_totais['id_jogo'] = id_jogo
    
    df_totais = df_totais.set_index('id_jogo')
    
    return df_mandante, df_visitante, df_totais

### Função que coleta todas as estatísticas SL Feminina 2018-2019

In [4]:
tabela_de_jogos = scrapper_tabela_jogos('http://cbv-web.dataproject.com/CompetitionMatches.aspx?ID=6')

In [5]:
tabela_de_jogos.head(2)

,data_jogo,local_jogo,mandante,sets_mandante,sets_visitante,visitante,rodada,turno
id_jogo,,,,,,,,
395,16/11/2018 - 19:30,PANELA DE PRESSAO,SESI VOLEI BAURU,3.0,0.0,HINODE BARUERI,1,Primeiro Turno
396,16/11/2018 - 20:00,HENRIQUE VILLABOIM,ESPORTE CLUBE PINHEIROS,3.0,1.0,FLUMINENSE F.C.,1,Primeiro Turno


In [6]:
stats = pd.DataFrame()
totais = pd.DataFrame()
falhas = []

for jogo in tabela_de_jogos.index:
    try:
        df1, df2, tots = estatisticas_jogo(jogo)
        aux = pd.concat([df1, df2])
        stats = pd.concat([stats, aux])

        totais = pd.concat([totais, tots])

        print(f"Estatísticas do jogo {jogo} coletadas...")
        
    except:
        falhas.append(jogo)
        print(f"Coleta do jogo {jogo} falhou!")

Estatísticas do jogo 395 coletadas...
Estatísticas do jogo 396 coletadas...
Estatísticas do jogo 397 coletadas...
Estatísticas do jogo 398 coletadas...
Estatísticas do jogo 399 coletadas...
Coleta do jogo 400 falhou!
Coleta do jogo 401 falhou!
Estatísticas do jogo 402 coletadas...
Estatísticas do jogo 403 coletadas...
Estatísticas do jogo 404 coletadas...
Estatísticas do jogo 405 coletadas...
Estatísticas do jogo 406 coletadas...
Estatísticas do jogo 407 coletadas...
Estatísticas do jogo 408 coletadas...
Estatísticas do jogo 409 coletadas...
Estatísticas do jogo 410 coletadas...
Estatísticas do jogo 411 coletadas...
Estatísticas do jogo 412 coletadas...
Estatísticas do jogo 413 coletadas...
Estatísticas do jogo 414 coletadas...
Estatísticas do jogo 415 coletadas...
Estatísticas do jogo 416 coletadas...
Estatísticas do jogo 417 coletadas...
Estatísticas do jogo 418 coletadas...
Estatísticas do jogo 419 coletadas...
Estatísticas do jogo 420 coletadas...
Estatísticas do jogo 421 coletadas

In [7]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4141 entries, (395, 'NAIANE RIOS ') to (620, 'BRUNA  COSTA')
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           2060 non-null   object
 1   set_1       2566 non-null   object
 2   set_2       2643 non-null   object
 3   set_3       2653 non-null   object
 4   set_4       1371 non-null   object
 5   set_5       477 non-null    object
 6   Pontos_Tot  4141 non-null   object
 7   BP          4141 non-null   object
 8   V-P         4141 non-null   object
 9   Saques_Tot  4141 non-null   object
 10  Saques_Err  4141 non-null   object
 11  Saques_Ace  4141 non-null   object
 12  Rec_Tot     4141 non-null   object
 13  Rec_Err     4141 non-null   object
 14  Rec_%Pos    4141 non-null   object
 15  Rec_%_Exc.  4141 non-null   object
 16  At_Tot      4141 non-null   object
 17  At_Err      4141 non-null   object
 18  At_Blk      4141 non-null   object
 19  At_Exc.    

In [8]:
stats.head()

# set_1 set_2 set_3 set_4 set_5 Pontos_Tot BP V-P  \
id_jogo Jogadora                                                               
395     NAIANE RIOS        3     *  None  None  None  None          -  -   -   
        GABRIELA CANDIDO   6  None  None     *  None  None          -  -   -   
        SARAELEN LIMA      7     1     5     1  None  None          5  1   -   
        EDINARA BRANCHER   9  None  None  None  None  None          -  -   -   
        TIFANNY ABREU     10     *  None  None  None  None          -  -  -1   

                         Saques_Tot  ... Rec_Err Rec_%Pos Rec_%_Exc. At_Tot  \
id_jogo Jogadora                     ...                                      
395     NAIANE RIOS               1  ...       -        .          .      -   
        GABRIELA CANDIDO          2  ...       -        .          .      -   
        SARAELEN LIMA             7  ...       -        .          .      8   
        EDINARA BRANCHER          -  ...       -        .          .      -   
        TIFANNY ABREU             -  ...       -        .          .      1   

                         At_Err At_Blk At_Exc. % Exc.  Block_Pts  \
id_jogo Jogadora                                                   
395     NAIANE RIOS           -      -       -       .         -   
        GABRIELA CANDIDO      -      -       -       .         -   
        SARAELEN LIMA         1      3       3     38%         2   
        EDINARA BRANCHER      -      -       -       .         -   
        TIFANNY ABREU         1      -       -      0%         -   

                                      time  
id_jogo Jogadora                            
395     NAIANE RIOS       SESI VOLEI BAURU  
        GABRIELA CANDIDO  SESI VOLEI BAURU  
        SARAELEN LIMA     SESI VOLEI BAURU  
        EDINARA BRANCHER  SESI VOLEI BAURU  
        TIFANNY ABREU     SESI VOLEI BAURU  

[5 rows x 23 columns]

In [9]:
falhas

[400, 401, 448, 537, 635, 593, 601, 602, 621, 622]

In [10]:
tabela_de_jogos.to_csv('data/tabela_superliga_fem_18_19.csv')
stats.to_csv('data/stats_superliga_fem_18_19.csv')